In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from typing import Tuple, Dict
from sklearn.linear_model import LassoCV

In [62]:


DATA_PATH = "data/spx_data_weekly.xlsx"

spx_df = pd.read_excel(DATA_PATH,sheet_name="spx data",header=[0, 1],index_col=0)
sector_df = pd.read_excel(DATA_PATH,sheet_name="sector data",header=[0, 1],index_col=0)
additional_df = pd.read_excel(DATA_PATH,sheet_name="additional data",header=[0, 1],index_col=0)

# Ensure dates sorted
spx_df = spx_df.sort_index()
sector_df = sector_df.sort_index()
additional_df = additional_df.sort_index()

# Convenience views
px_spx = spx_df.xs("PX_LAST", axis=1, level=1)          # stock prices
dy_spx = spx_df.xs("EQY_DVD_YLD_IND", axis=1, level=1)  # stock dividend yield
pe_spx = spx_df.xs("PE_RATIO", axis=1, level=1)         # PE Ratio

px_sector = sector_df.xs("PX_LAST", axis=1, level=1)    # sector prices
px_add = additional_df.xs("PX_LAST", axis=1, level=1)   # SPY, IEF, etc.

# Weekly returns
ret_spx = px_spx.pct_change()
ret_sector = px_sector.pct_change()
ret_add = px_add.pct_change()

spy_ret = ret_add["SPY"]  # market


print(dy_spx.loc["2015-07-24", :].nlargest(71))


DD      23.3362
KDP     19.3963
T       13.2744
BX      12.6662
OKE     12.3938
         ...   
PSA      5.0254
ES       5.0145
PAYX     5.0132
EQR      4.9816
KLAC     4.9253
Name: 2015-07-24 00:00:00, Length: 71, dtype: float64


<p style="font-size:28px;">1. Data Processing </p>

In [4]:


MIN_WEEKS_5Y = 52 * 5


def max_consecutive_non_nan(series: pd.Series) -> int:

    mask = series.notna().to_numpy()
    if not mask.any():
        return 0

    # Run-length encoding on boolean array
    max_run = 0
    current = 0
    for v in mask:
        if v:
            current += 1
            max_run = max(max_run, current)
        else:
            current = 0
    return max_run


def filter_tickers_with_5y_prices(price_df: pd.DataFrame,min_weeks: int = MIN_WEEKS_5Y) -> pd.Index:

    keep = []
    for ticker in price_df.columns:
        series = price_df[ticker]
        max_run = max_consecutive_non_nan(series)
        if max_run >= min_weeks:
            keep.append(ticker)
    return pd.Index(keep)


tickers_5y = filter_tickers_with_5y_prices(px_spx)
px_spx = px_spx[tickers_5y]
dy_spx = dy_spx[tickers_5y]
ret_spx = ret_spx[tickers_5y]

print(f"Tickers with >= 5 years continuous weekly data: {len(tickers_5y)}")



Tickers with >= 5 years continuous weekly data: 485


<p style="font-size:28px;">1.1. Data Processing </p>

In [6]:
results = []

# assuming valid_dates is built from the full panel, e.g.:
valid_dates = dy_spx.index

for t in valid_dates:
    # For each date, get the row of dividend yields and drop NaNs
    row = dy_spx.loc[t].dropna()   # <-- use full-data panel here

    # Ticker with highest and lowest yield on that date
    top = row.idxmax()
    bot = row.idxmin()

    results.append(
        {
            "date": t,
            "highest_ticker": top,
            "highest_yield": row[top],
            "lowest_ticker": bot,
            "lowest_yield": row[bot],
        }
    )

df_results = pd.DataFrame(results)

# Row with highest yield across ALL dates/tickers (panel-wide max)
row_highest = df_results.loc[df_results["highest_yield"].idxmax()]

# Row with lowest yield across ALL dates/tickers (panel-wide min)
row_lowest = df_results.loc[df_results["lowest_yield"].idxmin()]

print(
    f'Highest-yielding stock in the entire panel: '
    f'{row_highest["highest_ticker"]} on {row_highest["date"]}'
)
print(f'Dividend Yield: {row_highest["highest_yield"]:.4}\n')

print(
    f'Lowest-yielding stock in the entire panel: '
    f'{row_lowest["lowest_ticker"]} on {row_lowest["date"]}'
)
print(f'Dividend Yield: {row_lowest["lowest_yield"]:.4}')


Highest-yielding stock in the entire panel: TRGP on 2020-04-03 00:00:00
Dividend Yield: 64.8

Lowest-yielding stock in the entire panel: COO on 2021-09-03 00:00:00
Dividend Yield: 0.0132


In [50]:

dy_trailing_52 = dy_spx.rolling(window=52, min_periods=52).mean()
valid_dates = dy_trailing_52.dropna(how="all").index

last_date = dy_trailing_52.dropna(how="all").index[-1]
dy_last = dy_trailing_52.loc[last_date].dropna()

highest = dy_last.idxmax()
lowest = dy_last.idxmin()

print(f"Highest dividend-yielding stock (52-week average): {highest}")
print(f"Dividend Yield: {dy_last[highest]:.4f}\n")

print(f"Lowest dividend-yielding stock (52-week average): {lowest}")
print(f"Dividend Yield: {dy_last[lowest]:.4f}")


Highest dividend-yielding stock (52-week average): MO
Dividend Yield: 7.9489

Lowest dividend-yielding stock (52-week average): NVDA
Dividend Yield: 0.0318


**Was it driven more by changes in D or P?**</br>
TRGP's extreme yield is Price driven. COO’s small-yield is dividend driven, not price-driven. It pays negligible dividends, so the yield rounds down to zero. MO's high dividend yield is fundamentally dividend driven because the dividend payout is extremely large. NVDA's extremely low dividend yield is Price driven because the dividend is tiny and basically irrelevant.


<p style="font-size:28px;">1.2. A Carry Strategy & 1.3. Long-Short </p>

In [64]:


def top_20pct_dy_index(row: pd.Series, top_frac: float = 0.2):
    row = row.dropna()
    n = len(row)
    num =int(np.floor(top_frac * n))
    k_top = max(1,num)
    top_vals = row.nlargest(k_top)

    return top_vals.index

top_idx_per_date = dy_spx.apply(lambda row: top_20pct_dy_index(row, top_frac=0.2),axis=1)

prices_top20 = {}


for date, tickers in top_idx_per_date.items():
    k=len(tickers)
    prices_top20[date] = ret_spx.loc[date, tickers] 
    
prices_top20_df = pd.DataFrame.from_dict(prices_top20, orient="index")
prices_top20_df.index.name = "date"

row_means_top = prices_top20_df.mean(axis=1, skipna=True)
row_means_top.mean(skipna=True)



np.float64(0.0005570425510055037)

In [52]:

def bottom_20pct_dy_index(row: pd.Series, bottom_frac: float = 0.2):
    row = row.dropna()
    n = len(row)
    num =int(np.floor(bottom_frac * n))
    k_top = max(1,num)
    bottom_vals = row.nsmallest(k_top)

    return bottom_vals.index

bottom_idx_per_date = dy_spx.apply(lambda row: bottom_20pct_dy_index(row, bottom_frac=0.2),axis=1)


prices_bottom20 = {}


for date, tickers in bottom_idx_per_date.items():
    k=len(tickers)
   
    prices_bottom20[date] = ret_spx.loc[date, tickers] 

prices_bottom20_df = pd.DataFrame.from_dict(prices_bottom20, orient="index")
prices_bottom20_df.index.name = "date"

row_means_bottom = prices_bottom20_df.mean(axis=1, skipna=True)

ls_strat=row_means_top-row_means_bottom
ls_strat.mean(skipna=True)



np.float64(-0.004234821236993323)


<p style="font-size:28px;">1.4. Performance </p>

In [53]:

def portfolio_stats(ret: pd.Series,periods_per_year: int = 52) -> Dict[str, float]:
    """
    Compute annualized mean, vol, Sharpe, skew, VaR(5%), CVaR(5%), max drawdown.
    """
    ret = ret.dropna()

    mu_ann = ret.mean() * periods_per_year
    vol_ann = ret.std() * np.sqrt(periods_per_year)
    sharpe = mu_ann / vol_ann 

    skew = ret.skew()
    var_5 = ret.quantile(0.05)
    cvar_5 = ret[ret <= var_5].mean()

    value = (1 + ret).cumprod()
    peak = value.cummax()
    dd = (value / peak) - 1.0
    max_dd = dd.min()

    return {
        "ann_mean": mu_ann,
        "ann_vol": vol_ann,
        "sharpe": sharpe,
        "skew": skew,
        "VaR_5%": var_5,
        "CVaR_5%": cvar_5,
        "max_drawdown": max_dd,
    }


lo_ret = row_means_top          # long-only top 20% portfolio
ls_ret = ls_strat               # long–short (top – bottom)

spy_ret_aligned = spy_ret.loc[lo_ret.index]  # align with LO for comparison

lo_stats = portfolio_stats(lo_ret)
ls_stats = portfolio_stats(ls_ret)
spy_stats = portfolio_stats(spy_ret_aligned)


stats_df = pd.DataFrame(
    {
        "LO": lo_stats,
        "LS": ls_stats,
        "SPY": spy_stats,
    }
).T  # strategies as rows

stats_df



,ann_mean,ann_vol,sharpe,skew,VaR_5%,CVaR_5%,max_drawdown
LO,0.028966,0.218500,0.132569,-0.368940,-0.039660,-0.071357,-0.494195
LS,-0.220211,0.127726,-1.724086,0.071274,-0.031176,-0.044842,-0.899416
SPY,0.140826,0.173193,0.813116,-0.595027,-0.033571,-0.056766,-0.318291


<p style="font-size:28px;">2. Attribution </p>

In [ ]:

def ols_regression(
    y: pd.Series,
    X: pd.DataFrame,
) -> sm.regression.linear_model.RegressionResultsWrapper:
    """
    Run OLS of y on X with a constant.
    """
    combined = pd.concat([y, X], axis=1).dropna()
    y_clean = combined.iloc[:, 0]
    X_clean = combined.iloc[:, 1:]
    X_const = sm.add_constant(X_clean)
    model = sm.OLS(y_clean, X_const).fit()
    return model


# 2.1 – Market LFD: regress LO & LS on SPY

lo_ret_aligned = lo_ret.dropna()
ls_ret_aligned = ls_ret.dropna()
spy_for_lo = spy_ret.loc[lo_ret_aligned.index]
spy_for_ls = spy_ret.loc[ls_ret_aligned.index]

model_lo_spy = ols_regression(lo_ret_aligned, spy_for_lo.to_frame("SPY"))
model_ls_spy = ols_regression(ls_ret_aligned, spy_for_ls.to_frame("SPY"))

# Annualize alpha (weekly -> annual)
def annualize_alpha(
    model: sm.regression.linear_model.RegressionResultsWrapper,
    periods_per_year: int = 52,
) -> float:
    return model.params["const"] * periods_per_year


print("=== LO vs SPY ===")
print("alpha_ann:", annualize_alpha(model_lo_spy))
print("beta_SPY:", model_lo_spy.params["SPY"])
print("R2:", model_lo_spy.rsquared)

print("=== LS vs SPY ===")
print("alpha_ann:", annualize_alpha(model_ls_spy))
print("beta_SPY:", model_ls_spy.params["SPY"])
print("R2:", model_ls_spy.rsquared)

# 2.2 – Sector regression (LFD wrt sectors)
# Use sector ETF weekly returns
sector_rets = ret_sector  # columns: sector tickers

sector_for_lo = sector_rets.loc[lo_ret_aligned.index]
sector_for_ls = sector_rets.loc[ls_ret_aligned.index]

model_lo_sector = ols_regression(lo_ret_aligned, sector_for_lo)
model_ls_sector = ols_regression(ls_ret_aligned, sector_for_ls)

print("=== LO vs Sectors ===")
print("alpha_ann:", annualize_alpha(model_lo_sector))
print("R2:", model_lo_sector.rsquared)
print(model_lo_sector.params)

print("=== LS vs Sectors ===")
print("alpha_ann:", annualize_alpha(model_ls_sector))
print("R2:", model_ls_sector.rsquared)
print(model_ls_sector.params)

# 2.4 – MAG portfolio & two-factor regression (SPY + MAG)

MAG_TICKERS = ["AAPL", "MSFT", "GOOG", "AMZN", "NVDA", "META", "TSLA"]
common_mag = [t for t in MAG_TICKERS if t in ret_spx.columns]
mag_ret = ret_spx[common_mag].mean(axis=1)

mag_for_lo = mag_ret.loc[lo_ret_aligned.index]
mag_for_ls = mag_ret.loc[ls_ret_aligned.index]
spy_for_lo2 = spy_for_lo  # already aligned
spy_for_ls2 = spy_for_ls

X_lo_2f = pd.concat(
    [spy_for_lo2.rename("SPY"), mag_for_lo.rename("MAG")],
    axis=1,
)
X_ls_2f = pd.concat(
    [spy_for_ls2.rename("SPY"), mag_for_ls.rename("MAG")],
    axis=1,
)

model_lo_2f = ols_regression(lo_ret_aligned, X_lo_2f)
model_ls_2f = ols_regression(ls_ret_aligned, X_ls_2f)

print("=== LO vs SPY + MAG ===")
print("alpha_ann:", annualize_alpha(model_lo_2f))
print(model_lo_2f.params)
print("R2:", model_lo_2f.rsquared)

print("=== LS vs SPY + MAG ===")
print("alpha_ann:", annualize_alpha(model_ls_2f))
print(model_ls_2f.params)
print("R2:", model_ls_2f.rsquared)


=== LO vs SPY ===
alpha_ann: -0.00764408963822734
beta_SPY: 0.9585387007739592
R2: 0.6502900373100452
=== LS vs SPY ===
alpha_ann: -0.0217688309239376
beta_SPY: -0.10449807030491233
R2: 0.02307319974463462
=== LO vs Sectors ===
alpha_ann: 0.03067323056699773
R2: 0.9436097966920474
const    0.000590
XLK     -0.128193
XLI      0.112673
XLF      0.292003
XLC      0.087623
XLRE     0.318398
XLE      0.177915
XLY     -0.038016
XLB      0.110484
XLV     -0.074579
XLU      0.143052
XLP      0.017924
dtype: float64
=== LS vs Sectors ===
alpha_ann: 0.005551136144158756
R2: 0.6810891153892076
const    0.000107
XLK     -0.305949
XLI     -0.209150
XLF      0.237347
XLC      0.073642
XLRE     0.250666
XLE      0.177005
XLY     -0.150000
XLB     -0.010100
XLV     -0.314691
XLU      0.159482
XLP      0.084029
dtype: float64
=== LO vs SPY + MAG ===
alpha_ann: 0.07280162919942507
const    0.001400
SPY      1.567557
MAG     -0.484362
dtype: float64
R2: 0.7768486115628447
=== LS vs SPY + MAG ===
alpha_an

<p style="font-size:28px;">3. Dynamic hedged </p>

In [ ]:


ROLL_WIN = 52 * 5  # 5 years


def rolling_sector_betas(
    strat_ret: pd.Series,
    sector_ret: pd.DataFrame,
    window: int = ROLL_WIN,
) -> pd.DataFrame:
    """
    For each date t (after initial window),
    estimate betas from regression of strat_ret on sector_ret over past 'window' weeks.
    Returns a DataFrame of betas indexed by t (same as strat_ret index, truncated).
    """
    # align series and sector returns
    combined = pd.concat(
        [strat_ret.rename("strat"), sector_ret],
        axis=1,
    ).dropna()
    dates = combined.index
    cols_sector = sector_ret.columns

    betas = []

    for i in range(window, len(dates)):
        window_dates = dates[i - window : i]
        sub = combined.loc[window_dates]
        y = sub["strat"]
        X = sub[cols_sector]
        X_const = sm.add_constant(X)
        model = sm.OLS(y, X_const).fit()
        # store betas (exclude const)
        beta_t = model.params.drop("const")
        beta_t.name = dates[i]
        betas.append(beta_t)

    if not betas:
        return pd.DataFrame(columns=cols_sector)

    beta_df = pd.DataFrame(betas)
    beta_df = beta_df.reindex(columns=cols_sector)
    return beta_df


def sector_hedged_returns(
    strat_ret: pd.Series,
    sector_ret: pd.DataFrame,
    betas: pd.DataFrame,
) -> pd.Series:
    """
    Given strategy returns, sector returns, and rolling betas,
    compute hedged returns = strat_ret - sum_j beta_j(t-1)*sector_ret_j(t).
    We align on dates where betas are available.
    """
    common_dates = betas.index.intersection(sector_ret.index).intersection(
        strat_ret.index
    )
    common_dates = common_dates.sort_values()

    hedged = []

    for t in common_dates[1:]:
        # use beta from previous date as hedge ratio
        t_prev = common_dates[common_dates.get_loc(t) - 1]
        b = betas.loc[t_prev]  # betas at t_prev
        sector_t = sector_ret.loc[t, b.index]
        hedge = (b * sector_t).sum()
        hedged.append((t, strat_ret.loc[t] - hedge))

    return pd.Series(dict(hedged)).sort_index()


# Compute rolling betas and hedged returns for LO and LS strategies
lo_sector_ret = sector_rets.loc[lo_ret.index]
ls_sector_ret = sector_rets.loc[ls_ret.index]

lo_betas = rolling_sector_betas(lo_ret, lo_sector_ret)
ls_betas = rolling_sector_betas(ls_ret, ls_sector_ret)

lo_hedged = sector_hedged_returns(lo_ret, lo_sector_ret, lo_betas)
ls_hedged = sector_hedged_returns(ls_ret, ls_sector_ret, ls_betas)

lo_hedged_stats = portfolio_stats(lo_hedged)
ls_hedged_stats = portfolio_stats(ls_hedged)

print("LO sector-hedged stats:", lo_hedged_stats)
print("LS sector-hedged stats:", ls_hedged_stats)

# Sector LFD of hedged strategies (should have much lower R^2)
lo_hedged_model = ols_regression(
    lo_hedged,
    sector_rets.loc[lo_hedged.index],
)
ls_hedged_model = ols_regression(
    ls_hedged,
    sector_rets.loc[ls_hedged.index],
)

print("LO hedged vs sectors R2:", lo_hedged_model.rsquared)
print("LS hedged vs sectors R2:", ls_hedged_model.rsquared)


LO sector-hedged stats: {'ann_mean': np.float64(-0.020428350500080954), 'ann_vol': np.float64(0.05678025222066874), 'sharpe': np.float64(-0.35977914329596394), 'skew': np.float64(-0.07006092645003176), 'VaR_5%': np.float64(-0.01283681204353844), 'CVaR_5%': np.float64(-0.01671585836954045), 'max_drawdown': -0.08362163947434009}
LS sector-hedged stats: {'ann_mean': np.float64(-0.06710267962971729), 'ann_vol': np.float64(0.07622898018628653), 'sharpe': np.float64(-0.8802778085937053), 'skew': np.float64(0.06426242778012929), 'VaR_5%': np.float64(-0.018282011803087462), 'CVaR_5%': np.float64(-0.021617215817993852), 'max_drawdown': -0.14589358086068227}
LO hedged vs sectors R2: 0.3560757684325381
LS hedged vs sectors R2: 0.3487394737600158


<p style="font-size:28px;">4. Assessing the Forecast </p>

In [ ]:


# 4.1 – Forecast vs raw returns

def build_forecasts(
    dy_trailing: pd.DataFrame,
    top_frac: float = 0.2,
    bottom_frac: float = 0.2,
    pos_forecast: float = 0.001,
    neg_forecast: float = -0.001,
) -> pd.DataFrame:
    """
    For each date t, mark:
      +pos_forecast for top_frac names,
      neg_forecast for bottom_frac names,
      0 otherwise.
    """
    forecasts = pd.DataFrame(
        0.0, index=dy_trailing.index, columns=dy_trailing.columns
    )

    for t in dy_trailing.index:
        dy_t = dy_trailing.loc[t]
        dy_t = dy_t.dropna()
        if dy_t.empty:
            continue

        n = len(dy_t)
        k_top = max(1, int(np.floor(top_frac * n)))
        k_bot = max(1, int(np.floor(bottom_frac * n)))

        top = dy_t.nlargest(k_top).index
        bottom = dy_t.nsmallest(k_bot).index

        forecasts.loc[t, top] = pos_forecast
        forecasts.loc[t, bottom] = neg_forecast

    return forecasts


forecasts = build_forecasts(dy_trailing_52)

# Align forecasts at t with returns at t+1
common_dates = forecasts.index.intersection(ret_spx.index)
common_dates = common_dates.sort_values()

f_list = []
r_list = []

for i in range(len(common_dates) - 1):
    t = common_dates[i]
    t_next = common_dates[i + 1]
    f_t = forecasts.loc[t]
    r_tp1 = ret_spx.loc[t_next]

    # keep tickers with non-zero forecast and non-NaN return
    mask = (f_t != 0.0) & r_tp1.notna()
    if not mask.any():
        continue

    f_list.append(f_t[mask].to_numpy())
    r_list.append(r_tp1[mask].to_numpy())

if f_list:
    f_all = np.concatenate(f_list)
    r_all = np.concatenate(r_list)

    corr_fr = np.corrcoef(f_all, r_all)[0, 1]
    # R^2 from simple regression r = beta f (no intercept)
    beta = np.dot(f_all, r_all) / np.dot(f_all, f_all)
    r_hat = beta * f_all
    ss_res = np.sum((r_all - r_hat) ** 2)
    ss_tot = np.sum((r_all - r_all.mean()) ** 2)
    r2_forecast = 1 - ss_res / ss_tot

    print("Forecast vs raw returns corr:", corr_fr)
    print("Forecast vs raw returns R2:", r2_forecast)
else:
    print("No f_list – check data alignment for forecasts vs raw returns.")


# 4.2 – Forecast vs SPY-hedged residuals

def stock_betas_vs_spy(
    ret_stock: pd.DataFrame,
    spy_ret: pd.Series,
) -> pd.Series:
    """
    Estimate a single beta for each stock vs SPY over entire sample (simple).
    """
    betas = {}
    for ticker in ret_stock.columns:
        combined = pd.concat(
            [ret_stock[ticker].rename("r"), spy_ret.rename("spy")],
            axis=1,
        ).dropna()
        if len(combined) < 50:
            continue
        X = sm.add_constant(combined["spy"])
        model = sm.OLS(combined["r"], X).fit()
        betas[ticker] = model.params["spy"]
    return pd.Series(betas)


betas_spy = stock_betas_vs_spy(ret_spx, spy_ret)
betas_spy = betas_spy.reindex(ret_spx.columns).fillna(0.0)

# Compute residuals epsilon_{i,t} = r_{i,t} - beta_i * spy_t
eps_spx = ret_spx.sub(spy_ret, axis=0).mul(betas_spy, axis=1)  # wrong: need broadcasting carefully

# Correct residual: r_i,t - beta_i * spy_t
# We'll recompute properly:
eps_spx = ret_spx.copy()
for ticker in eps_spx.columns:
    eps_spx[ticker] = ret_spx[ticker] - betas_spy[ticker] * spy_ret

# Now align forecasts at t with eps at t+1
fe_list = []
e_list = []

for i in range(len(common_dates) - 1):
    t = common_dates[i]
    t_next = common_dates[i + 1]

    f_t = forecasts.loc[t]
    e_tp1 = eps_spx.loc[t_next]

    mask = (f_t != 0.0) & e_tp1.notna()
    if not mask.any():
        continue

    fe_list.append(f_t[mask].to_numpy())
    e_list.append(e_tp1[mask].to_numpy())

if fe_list:
    f_all = np.concatenate(fe_list)
    e_all = np.concatenate(e_list)

    corr_fe = np.corrcoef(f_all, e_all)[0, 1]
    beta_fe = np.dot(f_all, e_all) / np.dot(f_all, f_all)
    e_hat = beta_fe * f_all
    ss_res = np.sum((e_all - e_hat) ** 2)
    ss_tot = np.sum((e_all - e_all.mean()) ** 2)
    r2_fe = 1 - ss_res / ss_tot

    print("Forecast vs SPY-hedged residuals corr:", corr_fe)
    print("Forecast vs SPY-hedged residuals R2:", r2_fe)
else:
    print("No fe_list – check data alignment for forecasts vs residuals.")


# 4.3 – Forecast vs sector-hedged residuals

def stock_betas_vs_sectors(
    ret_stock: pd.DataFrame,
    ret_sector: pd.DataFrame,
) -> pd.DataFrame:
    """
    Estimate one beta vector per stock vs all sectors, using full sample.
    Returns DataFrame: index=tickers, columns=sector ETFs.
    """
    betas_dict = {}
    for ticker in ret_stock.columns:
        combined = pd.concat(
            [ret_stock[ticker].rename("r"), ret_sector],
            axis=1,
        ).dropna()
        if len(combined) < 50:
            continue
        y = combined["r"]
        X = combined[ret_sector.columns]
        X_const = sm.add_constant(X)
        model = sm.OLS(y, X_const).fit()
        betas_t = model.params.drop("const")
        betas_dict[ticker] = betas_t
    beta_df = pd.DataFrame(betas_dict).T
    return beta_df


beta_sectors = stock_betas_vs_sectors(ret_spx, ret_sector)
beta_sectors = beta_sectors.reindex(index=ret_spx.columns, columns=ret_sector.columns).fillna(0.0)

# Residuals u_{i,t} = r_{i,t} - sum_j beta_{ij} * sector_j,t
u_spx = pd.DataFrame(index=ret_spx.index, columns=ret_spx.columns, dtype=float)

for ticker in ret_spx.columns:
    r_i = ret_spx[ticker]
    b_i = beta_sectors.loc[ticker]
    # hedged component
    hedged_i = (ret_sector * b_i).sum(axis=1)
    u_spx[ticker] = r_i - hedged_i

fs_list = []
u_list = []

for i in range(len(common_dates) - 1):
    t = common_dates[i]
    t_next = common_dates[i + 1]

    f_t = forecasts.loc[t]
    u_tp1 = u_spx.loc[t_next]

    mask = (f_t != 0.0) & u_tp1.notna()
    if not mask.any():
        continue

    fs_list.append(f_t[mask].to_numpy())
    u_list.append(u_tp1[mask].to_numpy())

if fs_list:
    f_all = np.concatenate(fs_list)
    u_all = np.concatenate(u_list)

    corr_fu = np.corrcoef(f_all, u_all)[0, 1]
    beta_fu = np.dot(f_all, u_all) / np.dot(f_all, f_all)
    u_hat = beta_fu * f_all
    ss_res = np.sum((u_all - u_hat) ** 2)
    ss_tot = np.sum((u_all - u_all.mean()) ** 2)
    r2_fu = 1 - ss_res / ss_tot

    print("Forecast vs sector-hedged residuals corr:", corr_fu)
    print("Forecast vs sector-hedged residuals R2:", r2_fu)
else:
    print("No fs_list – check data alignment for forecasts vs sector residuals.")


Forecast vs raw returns corr: -0.008731231153207246
Forecast vs raw returns R2: -0.004712222544838829
Forecast vs SPY-hedged residuals corr: -0.00814064412373235
Forecast vs SPY-hedged residuals R2: 6.570341231737498e-05
Forecast vs sector-hedged residuals corr: -0.006056369966100719
Forecast vs sector-hedged residuals R2: -0.0002183342115611442


In [34]:


# 1. Choose a target asset
TARGET = "AAPL"

# 2. Build regression dataset for LASSO:
#    y_t = r_target,t ; X_t = returns of all OTHER stocks at t
ret_all = ret_spx.dropna(how="all")
y = ret_all[TARGET]
X = ret_all.drop(columns=[TARGET])

combined = pd.concat([y.rename("y"), X], axis=1).dropna()
y_clean = combined["y"].values
X_clean = combined.drop(columns=["y"]).values

print("LASSO sample shape:", X_clean.shape)  # (T, N-1)

# 3. Fit LASSO with CV (sparse replication portfolio)
lasso_cv = LassoCV(cv=5, random_state=0, max_iter=10000)
lasso_cv.fit(X_clean, y_clean)

alpha_opt = lasso_cv.alpha_
coef = pd.Series(lasso_cv.coef_, index=X.columns)
nonzero_weights = coef[coef != 0.0]

print("Optimal alpha:", alpha_opt)
print("Number of non-zero exposures:", len(nonzero_weights))

# In-sample fit quality
r2 = lasso_cv.score(X_clean, y_clean)
y_hat = lasso_cv.predict(X_clean)
residuals = y_clean - y_hat
print("R^2 of replication:", r2)
print("Tracking error (weekly std of residuals):", residuals.std())

# 4. Build forecast series for target vs replication portfolio

# Dates where we have trailing yield (for forecast) and returns
idx_common = dy_trailing_52.index.intersection(ret_all.index)
# Require AAPL to have a non-null trailing yield
mask_dates = idx_common[dy_trailing_52.loc[idx_common, TARGET].notna()]

# Target forecasts f_target(t) = forecast for AAPL at t
f_target = forecasts.loc[mask_dates, TARGET]

# Replication portfolio uses only non-zero LASSO weights
rep_tickers = nonzero_weights.index
w = nonzero_weights

# f_rep(t) = sum_j w_j * f_j(t)
f_rep = (forecasts.loc[mask_dates, rep_tickers] * w).sum(axis=1)

# 5. Compare the two forecast series
corr_forecast = np.corrcoef(f_target, f_rep)[0, 1]
diff = f_target - f_rep

print("Forecast correlation (target vs replication):", corr_forecast)
print("Mean forecast difference (weekly, target - rep):", diff.mean())
print("Std of forecast difference (weekly):", diff.std())

# Convert mean difference to annual % (since 0.001 ≈ 0.1% per week)
mean_diff_annual_pct = diff.mean() * 52 * 100
print("Mean forecast difference annualized (percentage points):",
      mean_diff_annual_pct)


LASSO sample shape: (260, 484)
Optimal alpha: 6.660302154539462e-05
Number of non-zero exposures: 59
R^2 of replication: 0.7321461946817996
Tracking error (weekly std of residuals): 0.020140311988592972
Forecast correlation (target vs replication): 0.7545083753136952
Mean forecast difference (weekly, target - rep): -0.0005700514056049021
Std of forecast difference (weekly): 0.00041049110072449143
Mean forecast difference annualized (percentage points): -2.964267309145491
